In [2]:
"""
Name - Matrikelnummer 
1) Pham, Ngoc Anh Trung - 7176267
2) Viktor Vironski - 4330455
3) Andy Disser - 5984875

Exercise Sheet 2
"""

from random import random
from sklearn import cluster, datasets
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import copy
import matplotlib.pyplot as plt
import matplotlib as mpl


def softmargin(data, labels, supportvec):
    """
    Compute the soft margin score

    Inputs:
    - data: the given data
    - labels: the given labels
    - supportvec: the given indices of support vectors

    Output:
    - the soft margin score ( (#correct - #mistakes)*margin )
    """

    # Left and right support vectors
    left = data[supportvec[0]]
    right = data[supportvec[1]]

    # Swap position if left > right
    if left > right:
        left, right = right, left

    # Compute threshold
    th = np.abs(np.divide(left+right,2))

    # Compute margin
    margin = np.abs(left-th)

    # Filter array for the points lying inside the margin
    inside_margin = (data-left >= 0) & (right-data >= 0)

    # Only consider the data that are inside the margin
    data_inside_margin = data[inside_margin]

    # The respective labels of the points inside the margin
    labels_inside_margin = labels[inside_margin]

    # Miss classification for label 1 when they are on the right side of the threshold
    missclassification1 = np.logical_and((data_inside_margin-th > 0), (labels_inside_margin == 0))
    # Miss classification for label 2 when they are on the left side of the threshold
    missclassification2 = np.logical_and((data_inside_margin-th < 0), (np.logical_or(labels_inside_margin == 1,\
         labels_inside_margin == 2)))

    # Miss classified when only one of the two criterion above is satisfied
    missclassification = np.logical_xor(missclassification1, missclassification2)

    # The number of missclassified points
    num_of_missclass = np.size(data_inside_margin[missclassification])

    # Correct classification criterion
    correctclassification1 = np.logical_and((data_inside_margin-th <= 0), (labels_inside_margin == 0))
    correctclassification2 = np.logical_and((data_inside_margin-th >= 0), np.logical_or(labels_inside_margin == 1,\
         labels_inside_margin == 2))

    # A point is correctly classified when one of the two criterion is satisfied
    correctclassification = np.logical_or(correctclassification1, correctclassification2)

    # The number of correct classified points
    num_of_correctclass = np.size(data_inside_margin[correctclassification])

    # Weight each of the miss classified with negative margin and correct classified with positive margin
    # return the sum as soft margin score
    return np.multiply((num_of_correctclass-num_of_missclass), margin)

In [4]:
# Import and access the data
iris = datasets.load_iris()
iris_data = iris['data']

# ============= Aufgabe 1b =================================

# Extract the petal widths
data = iris_data[:, 3]

# Create permutation of 0,1,..,149
permutation = np.random.permutation(np.arange(150))
train_filter = permutation < 100
test_filter = permutation >= 100

# Split the data, 100 samples for training, 50 samples for testing
train_data = data[train_filter]
test_data = data[test_filter]

# The respective labels
train_target = iris.target[train_filter]
test_target = iris.target[test_filter]

i = 0
best_margin_score = -np.inf
best_supp_vec_indices = np.zeros(2)
# Repeat 20 times
while i < 20:
    # Pick 2 different random integer from 0 to 99
    supp_vec_indices = np.random.choice(100, 2, replace=False)

    # Compute score
    score = softmargin(train_data, train_target, supp_vec_indices)

    # Compare score
    if score > best_margin_score:
        best_margin_score = score
        best_supp_vec_indices = supp_vec_indices

    i += 1

# ============ 1b Test the data ========================================

# Compute threshold
th = np.divide(train_data[best_supp_vec_indices[0]]+train_data[best_supp_vec_indices[1]], 2)

# filter the positions that the algorithm predicted the point belongs to class/label 1
predicted_label1 = th-test_data > 0

# filter the positions that the algorithm predicted the point belongs to class/label 2
predicted_label2 = th-test_data <= 0

# The truth label
truth_label1 = test_target == 0
truth_label2 = np.logical_or(test_target == 1, test_target == 2)

# Compare it with logical and
correctly_classified_1 = np.logical_and(predicted_label1, truth_label1)
correctly_classified_2 = np.logical_and(predicted_label2, truth_label2)

# Filter the correctly classified point
correct_classified_test_data = test_data[np.logical_or(correctly_classified_1, correctly_classified_2)]

print("Accuracy:", np.divide(np.size(correct_classified_test_data), np.size(test_data)))

Accuracy: 0.74
